In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

### Config the model from Google

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
output=model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


### Config the embedding model

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

c:\Navjyot Singh\AI\Agentic AI Training with Krish\GitCode\sunnyGit\Agentic-2.0\langraphEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384

## lets take a data embedd it and store in VDB

In [4]:
# Example: Load all .txt files from the "data" directory
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../data2", glob="**/*.txt")


In [5]:
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [6]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [7]:
new_docs=text_splitter.split_documents(documents=docs)

In [8]:
doc_string=[doc.page_content for doc in new_docs]

In [9]:
doc_string

['🇺🇸 Overview of the U.S. Economy The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under',
 'economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over',
 'and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade,',
 'of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and',
 'healthcare, finance, real estate, defense, and agriculture.',
 'U.S. GDP – Size, Composition, and Global Share As of 2024, the United States’ nominal GDP is estimated to be around $28 trillion

In [10]:
len(doc_string)

49

In [11]:
db=Chroma.from_documents(new_docs,embeddings)

In [12]:
retriever=db.as_retriever(search_kwargs={"k": 3})

In [13]:
retriever.invoke("industrial growth of usa?")

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='Future Outlook (2025–2030) Looking forward, the U.S. economy is expected to grow at a moderate pace, powered by innovation in AI, green energy, robotics, biotech, and quantum computing. The Biden'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='The U.S. maintains its GDP growth through strong innovation, entrepreneurship, and investment in R&D. With companies like Apple, Google, Amazon, Microsoft, and Tesla leading global markets, the U.S.'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='Final Summary The U.S. economy remains the engine of global growth, backed by unmatched innovation, financial dominance, and a strong institutional framework. Its $28 trillion GDP and influence over')]

## creation of pydantic class

In [14]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [15]:
from langchain.output_parsers import PydanticOutputParser

In [16]:
parser=PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [17]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

In [ ]:
'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

### this below agentstate is just for the explnation like how state works

In [18]:
Agentstate={}

In [25]:
Agentstate["messages"]=[]

In [26]:
Agentstate

{'messages': []}

In [27]:
Agentstate["messages"].append("hi how are you?")

In [28]:
Agentstate

{'messages': ['hi how are you?']}

In [29]:
Agentstate["messages"].append("what are you doing?")

In [30]:
Agentstate

{'messages': ['hi how are you?', 'what are you doing?']}

In [31]:
Agentstate["messages"].append("i hope everything fine")

In [32]:
Agentstate


{'messages': ['hi how are you?',
  'what are you doing?',
  'i hope everything fine']}

In [33]:
Agentstate["messages"][-1]

'i hope everything fine'

In [34]:
Agentstate["messages"][0]

'hi how are you?'

### this agentstate class you need to inside the stategraph

In [35]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [36]:
state={"messages":["hi"]}

In [37]:
state="hi"

In [48]:
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    
    print("Question",question)
    
    template="""
    Your task is to classify the given user query into one of the following categories: [USA,Web Search,Not Related]. 
    Only respond with the category name and nothing else.
    Before responding you will VERIFY the user query against the Verification Agent. In case the verification is false then you will again go back to the Superviros Agent and attempt to 
    classify the query again.
    If the query is not related to the categories, respond with "Not Related". 
    Ensure you only runt this not more than 2 times.

    User query: {question}
    {format_instructions}
    """
    
    prompt= PromptTemplate(
        template=template,
        input_variable=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    
    chain= prompt | model | parser
    
    response = chain.invoke({"question":question})
    
    print("Parsed response:", response)
    
    return {"messages": [response.Topic]}

In [49]:
state={"messages":["what is a today weather?"]}

In [29]:
state={"messages":["what is a GDP of usa??"]}

In [50]:
function_1(state)

Question what is a today weather?
Parsed response: Topic='Web Search' Reasoning='The query is a general weather inquiry that can be answered through a web search engine.'


{'messages': ['Web Search']}

In [41]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [ ]:
def router(state:AgentState):
    print("-> ROUTER ->")
    
    last_message=state["messages"][-1]
    print("last_message:", last_message)
    userQuery = last_message.lower()
    if "usa" in userQuery or "united states" in userQuery:
        route = "rag"
    elif  "search" in userQuery or "latest" in userQuery:
        route = "web"
    else:
        route = "llm"
    return {"route": route}

In [43]:
state={"messages":["search for todays weather details??"]}

In [44]:
router(state)

-> ROUTER ->
last_message: search for todays weather details??


{'route': 'web'}

workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG Call": "RAG"(function_2),
        "LLM Call": "LLM"(function_3),
    }
)

In [45]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [46]:
# RAG Function
def function_2(state:AgentState):
    print("-> RAG Call ->")
    
    question = state["messages"][0]
    
    prompt=PromptTemplate(
        template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, 
        just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
        
        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = rag_chain.invoke(question)
    return  {"messages": [result]}

In [ ]:
# LLM Function
def function_3(state:AgentState):
    print("-> LLM Call ->")
    question = state["messages"][0]
    
    # Normal LLM call
    complete_query = "Answer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = model.invoke(complete_query)
    return {"messages": [response.content]}

In [ ]:
# To install: pip install tavily-python
from tavily import TavilyClient
client = TavilyClient("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")
response = client.search(
    query="What is the price of Iphone16 in ",
    search_depth="basic",  # Options: 'basic', 'advanced' (more results, slower)
    include_answer=True,      # Tavily will generate a summarized answer
    include_raw_content=False # If you just want summaries, not full HTML/text
)
print(response)

In [51]:
# WEb CrawlerFunction

from tavily import TavilyClient

def function_4(state:AgentState):
    print("-> WEB CRAWLER Call ->")
    question = state["messages"][0]
    #question = "When was mahatma Gandhi born and where?"
    
    os.environ['TAVILY_TOKEN']=os.getenv("TAVILY_API_KEY")
    client = TavilyClient(os.environ['TAVILY_TOKEN'])
    # Normal LLM call
    complete_query = """Search online and return a concise, up-to-date answer to the following question:"""+ question + """
    Make sure to include:
    - A short summary or direct answer
    - At least 2-3 supporting sources (URLs)
    - Mention if there is no recent or relevant information
    "Following is the user question: """
    response = client.search(
    query=complete_query,
    search_depth="basic",  # Options: 'basic', 'advanced' (more results, slower)
    include_answer=True,      # Tavily will generate a summarized answer
    include_raw_content=False # If you just want summaries, not full HTML/text
    )
    #print(response)
    #return {"messages": [response]}
    return {"messages": [response["answer"]]}

In [52]:
state={"messages":["what is a GDP of usa??"]}

In [42]:
state["messages"][0]

'what is a GDP of usa??'

In [53]:
function_4(state)

-> WEB CRAWLER Call ->


{'messages': ["As of 2025, the U.S. GDP is a significant economic indicator, reflecting the nation's total economic output. The latest figures show a robust growth, highlighting the strength of the U.S. economy. For the most recent data, official government sources should be consulted."]}

In [54]:
from langgraph.graph import StateGraph,END

In [55]:

workflow3=StateGraph(AgentState)


In [56]:
workflow3.add_node("Supervisor",function_1)

In [57]:
workflow3.add_node("RAG",function_2)

In [77]:
workflow3.add_node("LLM",function_2)

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.


In [ ]:
workflow3.add_node("WEB",function_4)

In [59]:
workflow3.set_entry_point("Supervisor")

In [60]:
workflow3.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG Call": "RAG",
        "TAV Call": "TAV",
    }
)

In [61]:
workflow3.add_edge("RAG",END)
workflow3.add_edge("TAV",END)

In [62]:
app=workflow3.compile()

In [65]:
state={"messages":["What is the weather of Delhi today?"]}

In [64]:
state={"messages":["What is the capital of USA?"]}

In [66]:
print(state)

{'messages': ['What is the weather of Delhi today?']}


In [67]:
app.invoke(state)

Question What is the weather of Delhi today?
Parsed response: Topic='Not Related' Reasoning='The query asks about the weather in Delhi, India, which is not related to the USA.'
-> ROUTER ->
last_message: Not Related
-> WEB CRAWLER Call ->


{'messages': ['What is the weather of Delhi today?',
  'Not Related',
  "Today in Delhi, temperatures are soaring above 40°C with dry conditions and light winds. Humidity is low, and there's no rain expected. The heat index makes it feel even hotter."]}

In [ ]:
state={"messages":["what is a gdp of usa?"]}

In [ ]:
app.invoke(state)

In [ ]:
state={"messages":["can you tell me the industrial growth of world's most powerful economy?"]}

In [ ]:
state={"messages":["can you tell me the industrial growth of world's poor economy?"]}

In [ ]:
result=app.invoke(state)

In [ ]:
result["messages"][-1]

In [ ]:
Assignment:4
1. you have to create one supervisor node.
2. create one router function
3. create three more node
3.1 llm call (llm node)
3.2 RAG (rag node)
3.3 web crawler(fetch the info in realtime from internet)
4. created one more node after this for validation for generated output --> explore the validation part how to do that
5. if validation going to be failed in that case again go to supervioser node and then supervisor node will again decide what needs to be call next
6. once the validation will pass then only generate the final output

submission deadline till 9pm friday

submission instruction:
create your github repo and keep all the assisgnments over there(in that github repo)
i will share one googleform in group after completing your assignment you can share the github link through that google form. 

in next class will discuss about the 
1. react agent with multiple tool call
2.  agentic rag